## Load/import packages

In [1]:
import scipy
import numpy as np
import tensorflow as tf

from tensorflow.keras import Sequential, layers

# Check if Tensorflow uses GPU
print(tf.config.experimental.list_physical_devices("GPU"))

# Limit GPU memory usage
gpu_devices = tf.config.experimental.list_physical_devices("GPU")
for device in gpu_devices:
    tf.config.experimental.set_memory_growth(device, True)

print()
print(f"Tensorflow Version: {tf.__version__}")
print(f"Numpy Version: {np.__version__}")
print(f"Keras Version: {tf.keras.__version__}")

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

Tensorflow Version: 2.4.0-rc0
Numpy Version: 1.19.2
Keras Version: 2.4.0


# Build model

In [13]:
def build_model(inp_shape):
    model = Sequential(name="FW-RNN")

    model.add(layers.Dense(7, activation="softmax", name="Dense_Output"))
    model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["acc"])
    return model

In [12]:
cnn_rnn_LSTM = build_model((1, 1))
cnn_rnn_LSTM.summary()

Model: "RNN-LSTM"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
LSTM_1 (LSTM)                (None, 1, 16)             1152      
_________________________________________________________________
Dense_Output (Dense)         (None, 1, 7)              119       
Total params: 1,271
Trainable params: 1,271
Non-trainable params: 0
_________________________________________________________________


In [ ]:
# AW2_norm_minitrain = AW2_norm_minitrain.prefetch(buffer_size=tf.data.experimental.AUTOTUNE)
history = cnn_rnn_LSTM.fit(
    AW2_norm_train.batch(32, drop_remainder=True),
    validation_data=AW2_norm_val.batch(32, drop_remainder=True),
    epochs=10,
    verbose=1,
)

In [ ]:
# cnn_rnn_LSTM.evaluate(AW2_norm_train.batch(8, drop_remainder=True))

# Train + Evaluate model

# Predict on test set
1. Create loop which reads feature_data from each video directory
2. Predict on these features
3. Write predictions to filename.txt